<a href="https://colab.research.google.com/github/dorisratego/Combining-Excel-Sheets/blob/main/NLP/emotion_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [4]:
!wget --no-check-certificate --no-cache \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/binary-emotion.csv \
    -O /tmp/binary-emotion.csv

--2021-11-12 02:35:31--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/binary-emotion.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.128, 74.125.31.128, 74.125.141.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2690504 (2.6M) [text/csv]
Saving to: ‘/tmp/binary-emotion.csv’

/tmp/binary-emotion 100%[===================>]   2.57M  --.-KB/s    in 0.02s   

2021-11-12 02:35:31 (123 MB/s) - ‘/tmp/binary-emotion.csv’ saved [2690504/2690504]



In [5]:
from bs4 import BeautifulSoup
import string

stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at",
             "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do",
             "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having",
             "he", "hed", "hes", "her", "here", "heres", "hers", "herself", "him", "himself", "his", "how",
             "hows", "i", "id", "ill", "im", "ive", "if", "in", "into", "is", "it", "its", "itself",
             "lets", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought",
             "our", "ours", "ourselves", "out", "over", "own", "same", "she", "shed", "shell", "shes", "should",
             "so", "some", "such", "than", "that", "thats", "the", "their", "theirs", "them", "themselves", "then",
             "there", "theres", "these", "they", "theyd", "theyll", "theyre", "theyve", "this", "those", "through",
             "to", "too", "under", "until", "up", "very", "was", "we", "wed", "well", "were", "weve", "were",
             "what", "whats", "when", "whens", "where", "wheres", "which", "while", "who", "whos", "whom", "why",
             "whys", "with", "would", "you", "youd", "youll", "youre", "youve", "your", "yours", "yourself",
             "yourselves"]

table = str.maketrans('', '', string.punctuation)

In [6]:
#get Text from csv file
import csv
sentences=[]
labels=[]
with open('/tmp/binary-emotion.csv', encoding='UTF-8') as csvfile:
  reader = csv.reader(csvfile, delimiter=",")
  for row in reader:
    labels.append(int(row[0]))
    sentence = row[1].lower()
    sentence = sentence.replace(",", " , ")
    sentence = sentence.replace(".", " . ")
    sentence = sentence.replace("-", " - ")
    sentence = sentence.replace("/", " / ")
    soup = BeautifulSoup(sentence)
    sentence = soup.get_text()
    words = sentence.split()
    filtered_sentence = ""
    for word in words:
        word = word.translate(table)
        if word not in stopwords:
            filtered_sentence = filtered_sentence + word + " "
    sentences.append(filtered_sentence)
    
print(len(labels))
print(len(sentences))

35327
35327


In [7]:
#create training and test datasets
training_size = 28000

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [10]:
#use the tokenizer to create a vocabulary with up to 20,000 words
vocab_size = 20000
max_length = 10
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [11]:
print(training_sequences[0])
print(training_padded[0])

[18, 3257, 47, 4770, 613, 508, 951, 423]
[  18 3257   47 4770  613  508  951  423    0    0]


In [12]:
print(word_index)

{'<OOV>': 1, 'just': 2, 'not': 3, 'now': 4, 'day': 5, 'get': 6, 'no': 7, 'good': 8, 'like': 9, 'go': 10, 'dont': 11, 'work': 12, 'today': 13, 'got': 14, 'cant': 15, 'going': 16, 'one': 17, 'know': 18, 'back': 19, 'time': 20, 'love': 21, 'really': 22, 'will': 23, 'lol': 24, 'com': 25, 'u': 26, 'see': 27, 'want': 28, 'can': 29, 'home': 30, 'new': 31, 'think': 32, '2': 33, 'still': 34, 'oh': 35, 'miss': 36, 'much': 37, 'off': 38, 'need': 39, 'night': 40, 'last': 41, 'feel': 42, 'sad': 43, 'wish': 44, 'hope': 45, 'sorry': 46, 'bad': 47, 'didnt': 48, 'great': 49, 'morning': 50, 'thanks': 51, 'haha': 52, 'tomorrow': 53, 'twitter': 54, 'happy': 55, '3': 56, 'right': 57, 'tonight': 58, 'though': 59, 'gonna': 60, 'week': 61, 'fun': 62, 'make': 63, 'better': 64, 'weekend': 65, 'getting': 66, 'way': 67, 'nice': 68, 'come': 69, 'school': 70, 'sleep': 71, 'yeah': 72, 'hate': 73, 'people': 74, 'next': 75, 'days': 76, 'wont': 77, 'even': 78, 'never': 79, 'working': 80, 'say': 81, 'long': 82, 'wait': 